In [9]:
# Import the dependencies 
import pandas as pd
import gmaps
import requests

# Import the API key
from config import g_key

In [10]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [17]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [25]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights= [max(temp, 0) for temp in max_temp], dissipating= False, max_intensity= 300, point_radius= 4)

#Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig 

Figure(layout=FigureLayout(height='420px'))

In [26]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Heatmap of percent humidity 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig= gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating= False, max_intensity= 300, point_radius= 4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig= gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights= clouds, dissipating= False, max_intensity= 300, point_radius= 4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Heatmap of percent wind speed 
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig= gmaps.figure(center=(30.0, 31.0), zoom_level= 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights= wind, dissipating= False, max_intensity= 300, point_radius= 4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
# Ask the customer to add a minimum and maximum temperature value 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [31]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
10,10,Mareeba,AU,2022-09-25 00:10:49,-17.0000,145.4333,86.43,56,100,7.05
13,13,Port Blair,IN,2022-09-25 00:10:51,11.6667,92.7500,80.06,81,94,5.26
17,17,Lamu,KE,2022-09-25 00:10:53,-2.2717,40.9020,75.42,87,68,13.44
20,20,Kashan,IR,2022-09-25 00:10:55,33.9831,51.4364,81.59,16,0,2.06
35,35,Georgetown,MY,2022-09-25 00:07:45,5.4112,100.3354,76.95,93,40,2.30


In [32]:
preferred_cities_df.count()

City_ID       166
City          166
Country       166
Date          166
Lat           166
Lng           166
Max Temp      166
Humidity      166
Cloudiness    166
Wind Speed    166
dtype: int64

In [34]:
# 6.5.4 
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Mareeba,AU,86.43,-17.0000,145.4333,
13,Port Blair,IN,80.06,11.6667,92.7500,
17,Lamu,KE,75.42,-2.2717,40.9020,
20,Kashan,IR,81.59,33.9831,51.4364,
35,Georgetown,MY,76.95,5.4112,100.3354,


In [35]:
# Set the parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [38]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
10,Mareeba,AU,86.43,-17.0000,145.4333,Trinity Plains Tourist Park
13,Port Blair,IN,80.06,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
17,Lamu,KE,75.42,-2.2717,40.9020,Lamu House
20,Kashan,IR,81.59,33.9831,51.4364,Manouchehri Traditional House
35,Georgetown,MY,76.95,5.4112,100.3354,Cititel Penang


In [40]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [42]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [43]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))